# Language Modeling
* Machine Translation
* Document Summarization
* Topic Modeling
* Document Similarity
* Text Generation

In [ ]:
#Machine Translation
#Reference https://github.com/EricSchles/RNN-data-gen/blob/master/fully_connected_dnn_training.py